# MODULES

In [5]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

# CLASESS

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

## Euclidean distance

$d_E(p,q)=\sqrt{(p_x-q_x)^2+(p_y-q_y)^2}$

In [7]:
def get_euclidean_distance(p,q):
  """
  Arguments:
    p: [x,y] values for the starting point
    q: [x,y] values for the ending point
  """
  
  distance = np.sqrt(np.square(p[0]-q[0]) + np.square(p[1]-q[1]))

  return distance

## Mean Square Displacement


$MSD = \frac{1}{N-n} \sum \limits_{i=1}^{N-n}(\vec{r}_{i+n}-\vec{r}_i)^2 \quad\quad n=1,...,N-1$
<br><br>

$MSD = \frac{1}{N-n}\sum \limits_{i=1}^{N-n}{d_E(p,q)}^2 \quad\quad n=1,...,N-1$

In [8]:
def get_msd(tau,path):
  """
  Arguments:
    tau:
    path:
  """

  square_displacement = 0 

  for i in range(tau,path.shape[0],1):
    square_displacement += np.square(get_euclidean_distance(path.iloc[i-tau], path.iloc[i]))

  msd= (1/(path.shape[0]-tau))*square_displacement

  return msd

## Turning Angle

$
tan(\phi)=\frac{|\vec{p}\times\vec{q}|}{\vec{p}\cdot\vec{q}}
$

In [10]:
def get_turning_angle(a,b,c,round_to_zero = False):
  """
  Arguments:
    a: coordinates for p vector's tail
    b: coordinates p vector's head / q vector's tail
    c: coordinates q vector's head
    round_to_zero: if true checks if value is close enough to zero to be take as it
  """

  p = np.subtract(b,a)
  q = np.subtract(c,b)

  
  pq_cross = np.cross(p,q)
  pq_scalar = np.dot(p,q)

  phi_angle = np.arctan2(pq_cross,pq_scalar)

  if round_to_zero:
    if  count_like_zero(phi_angle):
      phi_angle = 0
      
  return phi_angle

## Custom round

In [11]:
def count_like_zero(number):
  """
  Arguments:
    number: 
  Return: True if -0.009 <= number <= 0.009 False otherwise
  """
  is_zero = False

  if  ((number >= -0.009) & (number <= 0)) | ((number >= 0) & (number <= 0.009)):
    is_zero = True

  return is_zero

## Is Negligible turn

In [12]:
def drop_turn(angle_i, angle_j):
  """
  Arguments:
    angle_i:
    angle_j: 
  Return: True if difference between angles is less than 0.001 False otherwise
  """
  is_dropable = False

  if (angle_i > 0) & (angle_j > 0): 
    rest = abs(angle_i - angle_j)
    if abs(angle_i - angle_j) < 0.001:
      is_dropable = True

  return is_dropable